# 6 deploy containers to turtlebot3

## change ${PJ_ROOT} to your directory.

In [ ]:
export PJ_ROOT="${HOME}/container-centric-fiware-demonstration"
cd ${PJ_ROOT};pwd

example)
```
/Users/user/container-centric-fiware-demonstration
```

## load environment variables

In [ ]:
source ${PJ_ROOT}/docs/azure_aks/env

## set apply and delete topic

In [ ]:
export MQTT_APPLY_TOPIC="/mqtt-kube-operator/apply"
export MQTT_DELETE_TOPIC="/mqtt-kube-operator/delete"

## set private repository 

In [ ]:
export REPOSITORY=$(az acr show --resource-group ${AKS_RG} --name ${ACR_NAME} | jq '.loginServer' -r)
sed -i '' -e "s/<<REPOSITORY>>/${REPOSITORY}/" ${PJ_ROOT}/docs/azure_aks/env
echo ${REPOSITORY}

## set private registry to turtlebot3's minikube

### create service principal of Azure ACR

In [ ]:
ACR_ID=$(az acr show --resource-group ${AKS_RG} --name ${ACR_NAME} --query "id" --output tsv)
az ad sp create-for-rbac --scopes ${ACR_ID} --role Reader --name ${ACR_NAME}sp > /tmp/acrsp
export ACR_USERNAME=$(cat /tmp/acrsp | jq .appId -r);echo "ACR_USERNAME=${ACR_USERNAME}"
export ACR_PASSWORD=$(cat /tmp/acrsp | jq .password -r);echo "ACR_PASSWORD=${ACR_PASSWORD}"
rm /tmp/acrsp

* if service principal already exists, delete it

In [ ]:
EXIST_ACR_ID=$(az ad sp list --display-name k8saksacrsp --query "[0].appId" -o tsv)
az ad sp delete --id ${EXIST_ACR_ID}

### set Azure ACR as private registry

In [ ]:
echo "kubectl create secret docker-registry ${ACR_NAME} --docker-server ${ACR_NAME}.azurecr.io --docker-email ${EMAIL} --docker-username ${ACR_USERNAME} --docker-password ${ACR_PASSWORD}"

* create secret to minikube using above command

## prepare turtlebot3
You have to setup common procedure at first. And then, you have to setup either turtlebot3_simulator (A.) or actual turtlebot3 robot (B.).

### common procedure: deploy ros-master

In [ ]:
docker build -t ${REPOSITORY}/tech-sketch/ros-master:0.2.0 ros/ros-master

In [ ]:
az acr login --name ${ACR_NAME}

In [ ]:
docker push ${REPOSITORY}/tech-sketch/ros-master:0.2.0

#### apply Service

In [ ]:
mosquitto_pub -h mqtt.${DOMAIN} -p 8883 --cafile ${PJ_ROOT}/secrets/ca.crt -d -u iotagent -P ${VERNEMQ__iotagent} -t ${MQTT_APPLY_TOPIC} -f ${PJ_ROOT}/ros/ros-master/json/service.json

* check minikube of turtlebot3

```bash
turtlebot3@turtlebot3:~$ kubectl get services -l app=ros-master
NAME         TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)     AGE
ros-master   ClusterIP   None         <none>        11311/TCP   2m
```

#### apply Deployment 

In [ ]:
envsubst < ${PJ_ROOT}/ros/ros-master/json/deployment.json.template > /tmp/deployment.json
mosquitto_pub -h mqtt.${DOMAIN} -p 8883 --cafile ${PJ_ROOT}/secrets/ca.crt -d -u iotagent -P ${VERNEMQ__iotagent} -t ${MQTT_APPLY_TOPIC} -f /tmp/deployment.json
rm /tmp/deployment.json

* check minikube of turtlebot3

```bash
turtlebot3@turtlebot3:~$ kubectl get deployments -l app=ros-master
NAME         DESIRED   CURRENT   UP-TO-DATE   AVAILABLE   AGE
ros-master   1         1         1            1           20s
```

```bash
turtlebot3@turtlebot3:~$ kubectl get pods -l app=ros-master
NAME                         READY     STATUS    RESTARTS   AGE
ros-master-b479df4b5-2wq74   1/1       Running   0          22s
```

### common procedure:  deploy fiware-ros-turtlebot3-bridge

#### build container image

In [ ]:
docker build -t ${REPOSITORY}/tech-sketch/fiware-ros-turtlebot3-bridge:0.2.0 ros/fiware-ros-turtlebot3-bridge

In [ ]:
az acr login --name ${ACR_NAME}

In [ ]:
docker push ${REPOSITORY}/tech-sketch/fiware-ros-turtlebot3-bridge:0.2.0

#### apply Secret

In [ ]:
export MQTT_YAML_BASE64=$(cat << __EOS__ | envsubst | base64
mqtt:
  host: "mqtt.${DOMAIN}"
  port: 8883
  username: "ros"
  password: "${VERNEMQ__ros}"
  use_ca: true
__EOS__)
envsubst < ${PJ_ROOT}/ros/fiware-ros-turtlebot3-bridge/json/mqtt-yaml-secret.json.template > /tmp/mqtt-yaml-secret.json
mosquitto_pub -h mqtt.${DOMAIN} -p 8883 --cafile ${PJ_ROOT}/secrets/ca.crt -d -u iotagent -P ${VERNEMQ__iotagent} -t ${MQTT_APPLY_TOPIC} -f /tmp/mqtt-yaml-secret.json
rm /tmp/mqtt-yaml-secret.json

* check minikube of turtlebot3

```bash
turtlebot3@turtlebot3:~$ kubectl get secrets -l app=ros-bridge
NAME                 TYPE      DATA      AGE
ros-bridge-secrets   Opaque    1         35m
```

#### apply ConfigMap

In [ ]:
mosquitto_pub -h mqtt.${DOMAIN} -p 8883 --cafile ${PJ_ROOT}/secrets/ca.crt -d -u iotagent -P ${VERNEMQ__iotagent} -t ${MQTT_APPLY_TOPIC} -f ${PJ_ROOT}/ros/fiware-ros-turtlebot3-bridge/json/configmap.json

* check minikube of turtlebot3

```bash
turtlebot3@turtlebot3:~$ kubectl get configmaps -l app=ros-bridge
NAME                    DATA      AGE
ros-bridge-configmaps   2         33m
```

#### apply Service

In [ ]:
mosquitto_pub -h mqtt.${DOMAIN} -p 8883 --cafile ${PJ_ROOT}/secrets/ca.crt -d -u iotagent -P ${VERNEMQ__iotagent} -t ${MQTT_APPLY_TOPIC} -f ${PJ_ROOT}/ros/fiware-ros-turtlebot3-bridge/json/service.json

* check minikube of turtlebot3

```bash
turtlebot3@turtlebot3:~$ kubectl get services -l app=ros-bridge
NAME         TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)     AGE
ros-bridge   ClusterIP   None         <none>        11311/TCP   33m
```

#### apply Deployment

In [ ]:
envsubst < ${PJ_ROOT}/ros/fiware-ros-turtlebot3-bridge/json/deployment.json.template > /tmp/deployment.json
mosquitto_pub -h mqtt.${DOMAIN} -p 8883 --cafile ${PJ_ROOT}/secrets/ca.crt -d -u iotagent -P ${VERNEMQ__iotagent} -t ${MQTT_APPLY_TOPIC} -f /tmp/deployment.json
rm /tmp/deployment.json

* check minikube of turtlebot3

```bash
turtlebot3@turtlebot3:~$ kubectl get deployments -l app=ros-bridge
NAME         DESIRED   CURRENT   UP-TO-DATE   AVAILABLE   AGE
ros-bridge   1         1         1            1           31m
```

```bash
turtlebot3@turtlebot3:~$ kubectl get pods -l app=ros-bridge
NAME                          READY     STATUS    RESTARTS   AGE
ros-bridge-7fdcdc7558-j66ng   1/1       Running   0          31m
```

#### confirm log

```bash
turtlebot3@turtlebot3:~$ kubectl logs -f $(kubectl get pods -l app=ros-bridge -o template --template "{{(index .items 0).metadata.name}}")
```

### common procedure:  deploy fiware-ros-turtlebot3-operator

#### build container image

In [ ]:
docker build -t ${REPOSITORY}/tech-sketch/fiware-ros-turtlebot3-operator:0.2.0 ros/fiware-ros-turtlebot3-operator

In [ ]:
az acr login --name ${ACR_NAME}

In [ ]:
docker push ${REPOSITORY}/tech-sketch/fiware-ros-turtlebot3-operator:0.2.0

#### apply ConfigMap

In [ ]:
mosquitto_pub -h mqtt.${DOMAIN} -p 8883 --cafile ${PJ_ROOT}/secrets/ca.crt -d -u iotagent -P ${VERNEMQ__iotagent} -t ${MQTT_APPLY_TOPIC} -f ${PJ_ROOT}/ros/fiware-ros-turtlebot3-operator/json/configmap.json

* check minikube of turtlebot3

```bash
turtlebot3@turtlebot3:~$ kubectl get configmaps -l app=turtlebot3-operator
NAME                             DATA      AGE
turtlebot3-operator-configmaps   1         18s
```

#### apply Service

In [ ]:
mosquitto_pub -h mqtt.${DOMAIN} -p 8883 --cafile ${PJ_ROOT}/secrets/ca.crt -d -u iotagent -P ${VERNEMQ__iotagent} -t ${MQTT_APPLY_TOPIC} -f ${PJ_ROOT}/ros/fiware-ros-turtlebot3-operator/json/service.json

* check minikube of turtlebot3

```bash
turtlebot3@turtlebot3:~$ kubectl get services -l app=turtlebot3-operator
NAME                  TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)     AGE
turtlebot3-operator   ClusterIP   None         <none>        11311/TCP   15s
```

#### apply Deployment 

In [ ]:
envsubst < ${PJ_ROOT}/ros/fiware-ros-turtlebot3-operator/json/deployment.json.template > /tmp/deployment.json
mosquitto_pub -h mqtt.${DOMAIN} -p 8883 --cafile ${PJ_ROOT}/secrets/ca.crt -d -u iotagent -P ${VERNEMQ__iotagent} -t ${MQTT_APPLY_TOPIC} -f /tmp/deployment.json
rm /tmp/deployment.json

* check minikube of turtlebot3

```bash
turtlebot3@turtlebot3:~$ kubectl get deployments -l app=turtlebot3-operator
NAME                  DESIRED   CURRENT   UP-TO-DATE   AVAILABLE   AGE
turtlebot3-operator   1         1         1            0           1m
```

```bash
turtlebot3@turtlebot3:~$ kubectl get pods -l app=turtlebot3-operator
NAME                                   READY     STATUS    RESTARTS   AGE
turtlebot3-operator-85b8bbbb88-kwstt   1/1       Running   0          32s
```

#### confirm log

```bash
turtlebot3@turtlebot3:~$ kubectl logs -f $(kubectl get pods -l app=turtlebot3-operator -o template --template "{{(index .items 0).metadata.name}}")
```

### B. actual turtlebot3 robot: deploy turtlebot3

#### set ROS workspace of turtlebot3

In [ ]:
export TURTLEBOT3_WORKSPACE=/home/turtlebot3/catkin_ws

#### build container image

In [ ]:
docker build -t ${REPOSITORY}/tech-sketch/turtlebot3-bringup:0.2.0 ros/turtlebot3_bringup

In [ ]:
az acr login --name ${ACR_NAME}

In [ ]:
docker push ${REPOSITORY}/tech-sketch/turtlebot3-bringup:0.2.0

#### apply Service

In [ ]:
mosquitto_pub -h mqtt.${DOMAIN} -p 8883 --cafile ${PJ_ROOT}/secrets/ca.crt -d -u iotagent -P ${VERNEMQ__iotagent} -t ${MQTT_APPLY_TOPIC} -f ${PJ_ROOT}/ros/turtlebot3_bringup/json/service.json

* check minikube of turtlebot3

```bash
turtlebot3@turtlebot3:~$ kubectl get services -l app=turtlebot3-bringup
NAME                 TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)     AGE
turtlebot3-bringup   ClusterIP   None         <none>        11311/TCP   13s
```

#### apply Deployment 

In [ ]:
envsubst < ${PJ_ROOT}/ros/turtlebot3_bringup/json/deployment.json.template > /tmp/deployment.json
mosquitto_pub -h mqtt.${DOMAIN} -p 8883 --cafile ${PJ_ROOT}/secrets/ca.crt -d -u iotagent -P ${VERNEMQ__iotagent} -t ${MQTT_APPLY_TOPIC} -f /tmp/deployment.json
rm /tmp/deployment.json

* check minikube of turtlebot3

```bash
turtlebot3@turtlebot3:~$ kubectl get deployments -l app=turtlebot3-operator
NAME                  DESIRED   CURRENT   UP-TO-DATE   AVAILABLE   AGE
turtlebot3-operator   1         1         1            0           1m
```

```bash
turtlebot3@turtlebot3:~$ kubectl get pods -l app=turtlebot3-operator
NAME                                   READY     STATUS    RESTARTS   AGE
turtlebot3-operator-85b8bbbb88-kwstt   1/1       Running   0          32s
```

#### confirm log

```bash
turtlebot3@turtlebot3:~$ kubectl logs -f $(kubectl get pods -l app=turtlebot3-bringup -o template --template "{{(index .items 0).metadata.name}}")
```